In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import RandomSampler
from warmup_scheduler import GradualWarmupScheduler
import albumentations
from work.utils.dataset import RGB2YHUTransform, PandasDataset
from work.utils.models import EfficientNet
from work.utils.train import train_model
from work.utils.metrics import model_checkpoint
import random

In [2]:
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: '/home/woshington/Projects/Doutorado/work/efficientnet-b0-08094119.pth'
}

data_dir = '../../dataset'
images_dir = os.path.join(data_dir, 'tiles')

df_train = pd.read_csv(f"../data/train_5fold.csv")

In [3]:
seed = 42
shuffle = True
batch_size = 2
num_workers = 4
output_classes = 5
init_lr = 3e-4
warmup_factor = 10
warmup_epochs = 1
n_epochs = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
loss_function = nn.BCEWithLogitsLoss()

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)


Using device: cuda


In [4]:
from work.utils.dataset import RGB2YHVTransform

transforms = albumentations.Compose([
    RGB2YHVTransform(),
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
])

valid_transforms =albumentations.Compose([
    RGB2YHVTransform()
])

In [5]:
df_train.columns = df_train.columns.str.strip()

train_indexes = np.where((df_train['fold'] != 3))[0]
valid_indexes = np.where((df_train['fold'] == 3))[0]

train = df_train.loc[train_indexes]
valid = df_train.loc[valid_indexes]

train_dataset = PandasDataset(images_dir, train, transforms=transforms)
valid_dataset = PandasDataset(images_dir, valid, transforms=valid_transforms)

In [6]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=2, num_workers=num_workers, sampler = RandomSampler(train_dataset)
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=2, num_workers=num_workers, sampler = RandomSampler(valid_dataset)
)

In [7]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_classes,
    pre_trained_model=pretrained_model
)
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [8]:
optimizer = optim.Adam(model.parameters(), lr = init_lr / warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs - warmup_epochs)
scheduler = GradualWarmupScheduler(optimizer, multiplier = warmup_factor, total_epoch = warmup_epochs, after_scheduler=scheduler_cosine)
save_path = f'models/with-noise-yhv.pth'

In [9]:
train_model(
    model=model,
    epochs=n_epochs,
    optimizer=optimizer,
    scheduler=scheduler,
    train_dataloader=train_loader,
    valid_dataloader=valid_loader,
    checkpoint=model_checkpoint,
    device=device,
    loss_function=loss_function,
    path_to_save_metrics="logs/with-noise-yhv.txt",
    path_to_save_model=save_path
)

Epoch 1/15



100%|██████████| 903/903 [07:57<00:00,  1.89it/s]


metrics {'val_loss': 0.3711381, 'val_acc': {'mean': 37.24282548725605, 'std': 1.15247016491602, 'ci_5': 35.34626066684723, 'ci_95': 39.224377274513245}, 'val_kappa': {'mean': 0.6604293589486305, 'std': 0.01389070968864319, 'ci_5': 0.6375778317778442, 'ci_95': 0.6823712123870127}, 'val_f1': {'mean': 0.326611205637455, 'std': 0.011639576973464323, 'ci_5': 0.3072046637535095, 'ci_95': 0.34626040905714034}, 'val_recall': {'mean': 0.33108225437998773, 'std': 0.011058732013172359, 'ci_5': 0.31285790503025057, 'ci_95': 0.34895030409097666}, 'val_precision': {'mean': 0.39871926182508466, 'std': 0.01630618915746472, 'ci_5': 0.3723119616508484, 'ci_95': 0.42466580271720883}}
Salvando o melhor modelo... 0.0 -> 0.6604293589486305
Epoch 2/15



100%|██████████| 903/903 [07:34<00:00,  1.99it/s]


metrics {'val_loss': 0.3686064, 'val_acc': {'mean': 41.203213238716124, 'std': 1.1704267375126294, 'ci_5': 39.27977979183197, 'ci_95': 43.10249388217926}, 'val_kappa': {'mean': 0.7027116273573382, 'std': 0.014239227807903253, 'ci_5': 0.6793818828007175, 'ci_95': 0.7264697790666725}, 'val_f1': {'mean': 0.37948748674988747, 'std': 0.012718722960974137, 'ci_5': 0.35894930064678193, 'ci_95': 0.40065894573926925}, 'val_recall': {'mean': 0.3886094211935997, 'std': 0.011763332948187315, 'ci_5': 0.36998034715652467, 'ci_95': 0.40892868191003795}, 'val_precision': {'mean': 0.4709523431956768, 'std': 0.014160680777828777, 'ci_5': 0.4479375913739204, 'ci_95': 0.4939526543021202}}
Salvando o melhor modelo... 0.6604293589486305 -> 0.7027116273573382
Epoch 3/15



...

metrics {'val_loss': 0.6875027, 'val_acc': {'mean': 55.84753467440605, 'std': 1.2207332708048908, 'ci_5': 53.73961329460144, 'ci_95': 57.731299102306366}, 'val_kappa': {'mean': 0.7732416422836013, 'std': 0.013533738798728262, 'ci_5': 0.7502321313269255, 'ci_95': 0.7942126028484182}, 'val_f1': {'mean': 0.4965928558409214, 'std': 0.012320205775306434, 'ci_5': 0.4762835368514061, 'ci_95': 0.5158090978860855}, 'val_recall': {'mean': 0.4923291586637497, 'std': 0.012290326558030655, 'ci_5': 0.47165232598781587, 'ci_95': 0.512037169933319}, 'val_precision': {'mean': 0.5074684229791164, 'std': 0.012595033723088265, 'ci_5': 0.48634358644485476, 'ci_95': 0.5274286657571793}}
Epoch 14/15



100%|██████████| 903/903 [07:03<00:00,  2.13it/s]


metrics {'val_loss': 0.70701694, 'val_acc': {'mean': 55.72703610658645, 'std': 1.2216847955764818, 'ci_5': 53.684210777282715, 'ci_95': 57.72852897644043}, 'val_kappa': {'mean': 0.7698533355120811, 'std': 0.013661827399770135, 'ci_5': 0.7474491749687122, 'ci_95': 0.7916056310995245}, 'val_f1': {'mean': 0.494777637809515, 'std': 0.012265939325412548, 'ci_5': 0.4744821161031723, 'ci_95': 0.5142522543668747}, 'val_recall': {'mean': 0.49064872911572455, 'std': 0.012228644609554027, 'ci_5': 0.4701273009181023, 'ci_95': 0.5106257885694504}, 'val_precision': {'mean': 0.5052973806560039, 'std': 0.01252835718937515, 'ci_5': 0.4840704932808876, 'ci_95': 0.5249433249235153}}
Epoch 15/15



100%|██████████| 903/903 [07:03<00:00,  2.13it/s]


metrics {'val_loss': 0.7159609, 'val_acc': {'mean': 55.29096977114678, 'std': 1.2309620549232445, 'ci_5': 53.24099659919739, 'ci_95': 57.34071731567383}, 'val_kappa': {'mean': 0.7692383992615909, 'std': 0.01370252368190862, 'ci_5': 0.7463088087311143, 'ci_95': 0.790861715714273}, 'val_f1': {'mean': 0.49345261174440386, 'std': 0.012429264237152743, 'ci_5': 0.47333581298589705, 'ci_95': 0.5139328181743622}, 'val_recall': {'mean': 0.4896802387535572, 'std': 0.012476483548452492, 'ci_5': 0.46974306404590604, 'ci_95': 0.5099111109972}, 'val_precision': {'mean': 0.5033396010994912, 'std': 0.012625210477940421, 'ci_5': 0.48311374336481094, 'ci_95': 0.5240724086761475}}
